# Fitting Description

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


In [ ]:
!cat requirements.txt
!pip install -r requirements.txt

albumentations==0.5.2
tensorflow==2.5.0
numpy>=1.19.5
torch>=1.7.1
pandas>=1.2.4
torchvision>=0.8.2
opencv-python
PyYAML
tqdm==4.56.0
scikit-image
scikit-learn
scipy
matplotlib
python-json-logger>=0.1.11
jupyterlab
seaborn
grad-cam
ttach
transformers
omegaconfCollecting albumentations==0.5.2
     |████████████████████████████████| 72 kB 613 kB/s 
     |████████████████████████████████| 11.5 MB 15.9 MB/s 
     |████████████████████████████████| 72 kB 286 kB/s 
     |████████████████████████████████| 8.6 MB 76.4 MB/s 
     |████████████████████████████████| 1.7 MB 58.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.6 MB 49.3 MB/s 
     |████████████████████████████████| 74 kB 3.5 MB/s 
     |████████████████████████████████| 37.1 MB 83 kB/s 
     |████████████████████████████████| 948 kB 82.8 MB/s 
     |████████████████████████████████| 392 kB 60.4 MB/s 
     |█

In [ ]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


## Load Data

In [ ]:
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append('.')
from definitions import ROOT_DIR
from airotica.utils import read_image

In [ ]:
DATA_PATH = ROOT_DIR / 'data'
TRAIN_IMAGES = DATA_PATH / 'images'
DOWNLOADED_TRAIN_IMAGES = DATA_PATH / 'downloaded_images'

In [ ]:
train = pd.read_csv(DATA_PATH / 'train.csv')
train_url_only = pd.read_csv(DATA_PATH / 'train_url_only.csv')
train_url_loaded_images = pd.read_csv(DATA_PATH / 'train_loaded_images.csv')
test = pd.read_csv(DATA_PATH / 'test.csv')
sample_submission = pd.read_csv(DATA_PATH / 'sample_submission.csv')

In [ ]:
import os
from os import listdir
train_images = listdir(TRAIN_IMAGES)
guid_train_images = [f.split('.')[0] for f in train_images if os.path.getsize(f"{str(TRAIN_IMAGES / f)}") != 0]

In [ ]:
dummy = test[test.guid.isin(guid_train_images)]
test_only_description = test[~(test.guid.isin(guid_train_images))]
test_only_images = dummy[dummy.description.isna()]
test_images_and_description = dummy[~(dummy.description.isna())]
print('without image or description:', len(test_only_description[test_only_description.description.isna()]))
print('only description:', test_only_description.shape[0])
print('only images:', test_only_images.shape[0])
print('images and description:', test_images_and_description.shape[0])

without image or description: 0
only description: 106
only images: 547
images and description: 570


 ## FitImages

### Prepared Data

In [ ]:
train_url_loaded_images['typology'] = train_url_loaded_images.typology.replace(
    {'предметы прикладного искусства, быта и этнографии ': 'предметы прикладного искусства, быта и этнографии'}
    )

In [ ]:
train_labels = train.typology.unique()
typology_to_label = dict(zip(sorted(train_labels), range(len(train_labels))))

In [ ]:
train_labels_url = train_url_loaded_images[~(train_url_loaded_images.typology.isna())].typology.unique()
typology_to_label_url = dict(zip(sorted(train_labels_url), range(len(train_labels_url))))

In [ ]:
train_url_only_train_labels = train_url_loaded_images[train_url_loaded_images.typology.isin(typology_to_label.keys())]
print(len(train_url_only_train_labels))

175076


In [ ]:
dummy_train = train[train.guid.isin(guid_train_images)]
dummy_train.loc[:, 'add'] = 0
train_url_only_train_labels.loc[:, 'add'] = 1
full_train = pd.concat((dummy_train, train_url_only_train_labels.drop('url', axis=1)), axis=0)
full_train = full_train[~(full_train.typology.isna())]
full_train['label'] = full_train['typology'].map(typology_to_label)
full_train

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,guid,description,typology,add,label
0,c84c547b-c5c5-45cf-9199-736df1301124,Монета. Екатерина II. Две копейки. 1789 г.,предметы нумизматики,0,7
1,af6fb03f-3d31-484f-ba9d-51e7b4ef55b9,Владельческий конволют. Собрание сочинений. / ...,редкие книги,0,12
2,5ad52d30-8239-4b41-bd56-da99ab5a555b,"Медаль ВДНХ ""За успехи в народном хозяйстве СС...",предметы нумизматики,0,7
3,6ce2ecbe-80d2-45d1-8a4c-7599950a7792,Монета. Михаил Федорович. Копейка,предметы нумизматики,0,7
4,73a48c2d-12c3-40da-b071-f9abd5bed64b,Временное удостоверение №12849 Охотина Р.А. о ...,документы,0,1
...,...,...,...,...,...
175119,0313dde2-b5bd-4a1a-b7ef-63436ad975ad,Фото ч/б матов. групповое с фигурными краями ...,фотографии и негативы,1,14
175120,669ecd91-4197-4734-a9a9-7cb79d4af9d3,Открытое письмо «Петродворец. Вид на Аллею фо...,предметы печатной продукции,1,8
175121,07ca9b7c-efea-49e3-8db8-e69e9a079bb4,Фото черно-белое. Художник Алексей Як. Кольцов...,фотографии и негативы,1,14
175122,c0df1cb4-1ceb-47ea-873e-6d96fc5a1a95,Грампластинка.,прочие,1,11


In [ ]:
full_train.to_csv('./data/full_images_data.csv', index=False)

### Model

In [ ]:
import os
import sys
import torch
import yaml
import logging
import numpy as np
import random
from tqdm import tqdm
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

sys.path.append('.')
from definitions import ROOT_DIR
from airotica.utils import save_checkpoint, convert_dict_to_tuple
from airotica.models import load_model
from airotica.loss_function import get_loss
from airotica.optimizers import (
    get_optimizer,
    get_scheduler
)
from airotica.dataset import get_data_loaders
from airotica.train import train as train_model, validation
from airotica.airotica import detectron

%matplotlib inline
%load_ext autoreload
%autoreload 2

log = logging.getLogger(__name__)

{"asctime": "2021-07-31 15:57:35", "name": "matplotlib.pyplot", "filename": "pyplot.py", "levelname": "DEBUG", "message": "Loaded backend module://ipykernel.pylab.backend_inline version unknown."}
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
DATA_PATH = ROOT_DIR / 'data'
CONFIG_PATH = ROOT_DIR / 'config.yml'

In [ ]:
with open(CONFIG_PATH) as f:
    data = yaml.safe_load(f)
config = convert_dict_to_tuple(dictionary=data)

device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(f'device: {device_name}')

device: cuda


In [ ]:
seed = config.dataset.seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

os.environ['CUDA_VISIBLE_DEVICES'] = config.cuda_id

In [ ]:
ls EXPERIMENTS/resnext152/

model_0000_0.736543.pth  model_0004_0.754029.pth
model_0001_0.750158.pth  model_0007_0.755441.pth


In [ ]:
!pip uninstall pandas
!pip install pandas==1.1.5

Found existing installation: pandas 1.3.1
Uninstalling pandas-1.3.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pandas-1.3.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas/*
Proceed (y/n)? y
  Successfully uninstalled pandas-1.3.1
     |████████████████████████████████| 9.5 MB 9.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.


In [ ]:
detectron(config, 
          './sub/prediction_images.csv', 
          './EXPERIMENTS/resnext152/model_0007_0.755441.pth', 
          'resnext101_32x8d')

100%|██████████| 1223/1223 [00:29<00:00, 41.33it/s]


In [ ]:
images_prediction = pd.read_csv('./sub/prediction_images.csv')

In [ ]:
desc_prediction = pd.read_csv('./sub/distilbert.csv')

In [ ]:
from tensorflow.keras.models import load_model

"""
Preprocessing.
"""

import numpy as np
from tqdm import tqdm
from transformers import PreTrainedTokenizerFast


def preprocess(texts, tokenizer_path, max_len=32):

    input_ids, input_masks = [], []

    tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_path)
    tokenizer.mask_token = '[MASK]'
    tokenizer.pad_token = "[PAD]"
    tokenizer.sep_token = "[SEP]"
    tokenizer.cls_token = "[CLS]"
    tokenizer.unk_token = "[UNK]"

    for text in tqdm(texts):
        encoded = tokenizer.encode_plus(text,
                                        max_length=max_len,
                                        pad_to_max_length=True,
                                        truncation=True)
        input_ids.append(encoded['input_ids'])
        input_masks.append(encoded['attention_mask'])

    return [np.array(input_ids), np.array(input_masks)]


MODELS_PATH = ROOT_DIR / 'models'
TOKENIZERS_PATH = MODELS_PATH / 'tokenizers'
test_ = test[~(test.description.isna())]
categories = pd.read_csv('./data/categories.csv')['category'].tolist()

tokenizers = [str(TOKENIZERS_PATH / name) for name in ['wordpiece_70k.json', 'bpe_60k.json']]
models = ['./models/model/distilbert_wordpiece_70k', './models/model/distilbert_bpe_60k']

probs = []
item_name = test_.description
for model, tokenizer in zip(models, tokenizers):
    input = preprocess(item_name, tokenizer_path=tokenizer)
    mdl = load_model(model)
    proba = mdl.predict(input, batch_size=256, verbose=True)
    probs.append(proba)

  0%|          | 0/676 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 676/676 [00:00<00:00, 8261.39it/s]


3/3 [==============================] - 2s 75ms/step


100%|██████████| 676/676 [00:00<00:00, 7769.27it/s]


3/3 [==============================] - 1s 72ms/step


In [ ]:
pb = probs[0] + probs[1]
df_pb = pd.DataFrame(pb, columns=categories)

In [ ]:
pred = df_pb.idxmax(axis=1)

In [ ]:
len(test) - len(test_)

547

In [ ]:
lbl_to_typology = {v:k for k, v in typology_to_label.items()}

In [ ]:
im_test = images_prediction[images_prediction.sign_0 != -1]
im_test['result'] = im_test['sign_0'].map(lbl_to_typology)
sample_submission.loc[test_.index, 'typology'] = pred.values
sample_submission.loc[im_test.index, 'typology'] = im_test.result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
sample_submission.to_csv('./sub/img_with_distilbert.csv', index=False)

In [ ]:
detectron(config, 
          './sub/prediction_images_many.csv', 
          './EXPERIMENTS/resnext152/model_0007_0.755441.pth', 
          'resnext101_32x8d',
          17,
          experiment=True)

100%|██████████| 1223/1223 [05:49<00:00,  3.50it/s]


In [ ]:
images_prediction_2 = pd.read_csv('./sub/prediction_images_many.csv')
for i, row in images_prediction_2.iterrows():
    values, counts = np.unique(row[:17].astype(int), return_counts=True)
    if len(values) == 1:
        images_prediction_2.at[i, 'sign'] = values[0]
    else:
        images_prediction_2.at[i, 'sign'] = values[np.argmax(counts)]

In [ ]:
im_test_2 = images_prediction_2[images_prediction_2.sign_0 != -1]
im_test_2['result'] = im_test_2['sign'].map(lbl_to_typology)
sample_submission.loc[test_.index, 'typology'] = pred.values
sample_submission.loc[im_test.index, 'typology'] = im_test_2.result
sample_submission.to_csv('./sub/img_with_distilbert_3.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
images_prediction = pd.read_csv('./sub/prediction_images.csv')

In [ ]:
print("Loading model...")
net = load_model(config, 
                 device=device_name)
print("Done.")
criterion, criterion_val = get_loss(config, device=device_name)
optimizer = get_optimizer(config, 'SGD', net)

n_epoch = 20
scheduler = get_scheduler(config, optimizer)
train_epoch = tqdm(range(config.train.n_epoch),
                   dynamic_ncols=True,
                   desc='Epochs',
                   position=0)

Loading model...
{"asctime": "2021-07-31 14:15:46", "name": "airotica.models", "filename": "models.py", "levelname": "INFO", "message": "ResNext101_32x8d"}
Done.
{"asctime": "2021-07-31 14:15:48", "name": "airotica.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "0.002"}
{"asctime": "2021-07-31 14:15:48", "name": "airotica.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "Opt: SGD"}


Epochs:   0%|          | 0/21 [00:00<?, ?it/s]

### Data

In [ ]:
trained = pd.read_csv('./data/full_images_data.csv')

In [ ]:
label_to_typology = {v: k for k, v in typology_to_label.items()}

In [ ]:
for i in range(15):
    globals()[f'label_{i}'] = trained[trained['label'] == i]
    print(globals()[f'label_{i}'].shape[0], label_to_typology[i], i)

14027 графика 0
24186 документы 1
3365 живопись 2
695 оружие 3
22153 предметы археологии 4
4074 предметы естественнонаучной коллекции 5
1431 предметы минералогической коллекции 6
21596 предметы нумизматики 7
19836 предметы печатной продукции 8
21262 предметы прикладного искусства, быта и этнографии 9
1532 предметы техники 10
8244 прочие 11
7746 редкие книги 12
505 скульптура 13
28772 фотографии и негативы 14


In [ ]:
n = 1000
full = pd.concat((
     label_0[:n],
     label_1[:n],
     label_2[:n],
     label_3,
     label_4[:n],
     label_5[:n],
     label_6,
     label_7[:n],
     label_8[:n],
     label_9[:n],
     label_10,
     label_11[:n],
     label_12[:n],
     label_13,
     label_14[:n])
    )

In [ ]:
failed_guid = ['5c989b07-c6a2-4c7b-bd48-e6f9bca7eb27', 
               'cbd73791-e011-40e1-8cc7-f72a745bcebb', 
               'bebd7545-b84c-4af1-84bd-425e2b6313a9',
               'baa188a2-d630-489f-b92e-6cfaba762b0b',
               '4c39c840-cf9f-4363-82bf-74ea084c4f0d',
               '94c866c7-6230-409f-a5f9-b50f976ab62f',
               '46ecbdac-521d-4967-8d04-78e1811d3403',
               '60cacca9-e773-4932-83ca-e4d61bf5c134',
               '76975b28-9dfd-4cb6-bb54-c10a71b886a8',
               'b8165817-1662-4c17-ab53-fd0525537140',
               '99d35800-949b-445f-a018-d2e60b67d9c6',
               '852f75be-19bf-4148-932c-1637facbe8bc',
               'b8165817-1662-4c17-ab53-fd0525537140',
               'a733aa01-0e72-4a69-9045-6d262110c766',
               '78d9b613-9ced-4c65-9cb0-6b2d4dc8006c',
               '26e283c4-5919-4586-912c-3a45bfe290da']

full_train_2 = full[~(full.guid.isin(failed_guid))].reset_index(drop=True)

In [ ]:
X_train, X_valid = train_test_split(full_train_2, test_size=0.1, random_state=42)
print(f'train: {X_train.shape[0]}, valid: {X_valid.shape[0]}')

train: 13638, valid: 1516


In [ ]:
dt, dv = get_data_loaders(train_data=X_train, 
                          valid_data=X_valid, 
                          config=config, 
                          img_path=TRAIN_IMAGES, 
                          download_img_path=DOWNLOADED_TRAIN_IMAGES, 
                          use_data=True)

{"asctime": "2021-07-31 14:26:24", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Preparing train reader..."}
{"asctime": "2021-07-31 14:26:24", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Done."}
{"asctime": "2021-07-31 14:26:24", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "here"}
{"asctime": "2021-07-31 14:26:24", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Preparing valid reader..."}
{"asctime": "2021-07-31 14:26:24", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Done."}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
out_dir = str(ROOT_DIR / os.path.join(config.outdir, 'resnext152'))
print("Savedir: {}".format(out_dir))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

Savedir: /content/drive/MyDrive/digital_breakthrough/task_3/EXPERIMENTS/resnext152


In [ ]:
f1_best = 0
for epoch in train_epoch:
    train_model(net, dt, criterion, optimizer, config, epoch)
    f1_val = validation(net, dv, criterion_val, epoch)
    if f1_val > f1_best:
        save_checkpoint(net, optimizer, scheduler, epoch, out_dir, f1_val)
        f1_best = f1_val
    scheduler.step()


Train:   0%|          | 0/284 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))

Train: 100%|██████████| 284/284 [05:10<00:00,  1.09s/it]


Val:   0%|          | 0/32 [00:00<?, ?it/s]


    Train process of epoch: 0 is done;
    loss: 0.9632; acc: 0.71, f1: 0.70354180
    




Val:   3%|▎         | 1/32 [00:12<06:31, 12.62s/it]

Val:   6%|▋         | 2/32 [00:15<03:22,  6.75s/it]

Val:   9%|▉         | 3/32 [00:15<01:50,  3.81s/it]

Val:  12%|█▎        | 4/32 [00:15<01:08,  2.43s/it]

Val:  16%|█▌        | 5/32 [00:16<00:45,  1.67s/it]

Val:  19%|█▉        | 6/32 [00:16<00:31,  1.22s/it]

Val:  22%|██▏       | 7/32 [00:16<00:23,  1.08it/s]

Val:  25%|██▌       | 8/32 [00:17<00:17,  1.37it/s]

Val:  28%|██▊       | 9/32 [00:17<00:13,  1.65it/s]

Val:  31%|███▏      | 10/32 [00:17<00:11,  1.95it/s]

Val:  34%|███▍      | 11/32 [00:18<00:09,  2.23it/s]

Val:  38%|███▊      | 12/32 [00:18<00:08,  2.46it/s]

Val:  41%|████      | 13/32 [00:18<00:07,  2.62it/s]

Val:  44%|████▍     | 14/32 [00:19<00:06,  2.75it/s]

Val:  47%|████▋     | 15/32 [00:19<00:06,  2.69it/s]

Val:  50%|█████     | 16/32 [00:19<00:05,  2.84it/s]

Val:  53%|█████▎    | 17/32 [00:24<00:24,  1.64s/it]

Val:  56%|█████▋    | 18/32 [00:24<00:17,  1.24s/it]

Val:  59%|█████▉    | 19/32 [00:25<


        Validation of epoch: 0 is done; 
        loss: 0.8295; acc: 0.74, f1: 0.73654252
        



Train: 100%|██████████| 284/284 [05:05<00:00,  1.08s/it]


Val:   0%|          | 0/32 [00:00<?, ?it/s]

...


        Validation of epoch: 4 is done; 
        loss: 0.9559; acc: 0.75, f1: 0.75402884
        



Train: 100%|██████████| 284/284 [05:04<00:00,  1.07s/it]


Val:   0%|          | 0/32 [00:00<?, ?it/s]


    Train process of epoch: 5 is done;
    loss: 0.2391; acc: 0.95, f1: 0.95328969
    




Val:   3%|▎         | 1/32 [00:09<04:53,  9.47s/it]

Val:   6%|▋         | 2/32 [00:17<04:19,  8.63s/it]

Val:   9%|▉         | 3/32 [00:17<02:20,  4.85s/it]

Val:  12%|█▎        | 4/32 [00:18<01:25,  3.06s/it]

Val:  16%|█▌        | 5/32 [00:18<00:55,  2.07s/it]

Val:  19%|█▉        | 6/32 [00:18<00:38,  1.48s/it]

Val:  22%|██▏       | 7/32 [00:19<00:27,  1.11s/it]

Val:  25%|██▌       | 8/32 [00:19<00:20,  1.17it/s]

Val:  28%|██▊       | 9/32 [00:19<00:16,  1.43it/s]

Val:  31%|███▏      | 10/32 [00:20<00:13,  1.66it/s]

Val:  34%|███▍      | 11/32 [00:20<00:10,  1.93it/s]

Val:  38%|███▊      | 12/32 [00:20<00:09,  2.16it/s]

Val:  41%|████      | 13/32 [00:21<00:08,  2.34it/s]

Val:  44%|████▍     | 14/32 [00:21<00:07,  2.42it/s]

Val:  47%|████▋     | 15/32 [00:21<00:06,  2.61it/s]

Val:  50%|█████     | 16/32 [00:22<00:05,  2.73it/s]

Val:  53%|█████▎    | 17/32 [00:22<00:05,  2.84it/s]

Val:  56%|█████▋    | 18/32 [00:22<00:04,  2.86it/s]

Val:  59%|█████▉    | 19/32 [00:23<


        Validation of epoch: 5 is done; 
        loss: 1.0483; acc: 0.73, f1: 0.73462190
        
